In [1]:
import pandas as pd
import s3fs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Especificar la ruta del archivo Parquet en el bucket de S3
s3_path = 's3://agb-prueba-bucket/Covid Data.parquet'

# Leer el archivo Parquet desde S3
covid_data = pd.read_parquet(s3fs.S3FileSystem().open(s3_path))

In [2]:
covid_data.head(5)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [3]:
covid_data.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU'],
      dtype='object')

In [4]:
covid_data['ICU'].value_counts()

97    848544
2     175685
1      16858
99      7488
Name: ICU, dtype: int64

In [5]:
covid_data["DEATH"] = [1 if each == "9999-99-99" else 0 for each in covid_data.DATE_DIED]
covid_data.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU,DEATH
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,1,2,2,2,2,2,3,97,0
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,1,2,2,1,1,2,5,97,0
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,3,2,0
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,7,97,0
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,1,2,2,2,2,2,3,97,0


In [6]:
covid_data = covid_data.drop(labels=["DATE_DIED"],axis=1)

covid_data.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'INTUBED', 'PNEUMONIA',
       'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU', 'DEATH'],
      dtype='object')

In [7]:
X = covid_data.drop(labels=["DEATH"],axis=1)
Y = covid_data["DEATH"]

In [8]:
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))  # Capa de entrada
model.add(Dense(16, activation='relu'))  # Capa oculta
model.add(Dense(1, activation='sigmoid'))  # Capa de salida

2024-02-20 16:52:37.400289: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
26215/26215 [==============================] - 53s 2ms/step - loss: 0.1422 - accuracy: 0.9369
Epoch 2/10
26215/26215 [==============================] - 58s 2ms/step - loss: 0.1247 - accuracy: 0.9463
Epoch 3/10
26215/26215 [==============================] - 53s 2ms/step - loss: 0.1214 - accuracy: 0.9475
Epoch 4/10
26215/26215 [==============================] - 61s 2ms/step - loss: 0.1200 - accuracy: 0.9480
Epoch 5/10
26215/26215 [==============================] - 63s 2ms/step - loss: 0.1194 - accuracy: 0.9482
Epoch 6/10
26215/26215 [==============================] - 53s 2ms/step - loss: 0.1189 - accuracy: 0.9484
Epoch 7/10
26215/26215 [==============================] - 52s 2ms/step - loss: 0.1185 - accuracy: 0.9486
Epoch 8/10
26215/26215 [==============================] - 48s 2ms/step - loss: 0.1183 - accuracy: 0.9486
Epoch 9/10
26215/26215 [==============================] - 53s 2ms/step - loss: 0.1182 - accuracy: 0.9486
Epoch 10/10
26215/26215 [==============================

In [16]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo: {accuracy * 100.0}%')

6554/6554 [==============================] - 8s 1ms/step - loss: 0.1176 - accuracy: 0.9487
Precisión del modelo: 94.87161040306091%
